In [1]:
!pip install xgboost
!pip install joblib

In [2]:
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from numpy import loadtxt
import os
import scipy
import matplotlib.pyplot as plt
from matplotlib import rcParams
import warnings
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
import seaborn as sb

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb

from xgboost import XGBClassifier
#from sklearn.model_selection import train_test_split
import joblib
from joblib import dump, load
import pickle

In [3]:
#To view all columns in DF ,expanded without ellipses
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [4]:
df=pd.read_csv("NFL_datasetForML.csv")
df.head()

,Unnamed: 0,season,week,boxscore_id,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,rush_attempts,rush_yards,rush_tds,total_yards,first_downs,sacks,sacks_yards,pass_interceptions,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,roof,surface,temp,humidity,wind_chill,wind_mph,Score,Won
0,0,2002,1,200209050nyg,sfo,0,26,16,166.0,166,1.0,25,113.0,0.0,279,13,0,0,1.0,0,0,1,27:32:00,0,0,4,12,0,0,outdoors,grass,73,49.0,0,7,16,1
1,1,2002,1,200209050nyg,nyg,1,45,28,342.0,318,0.0,22,43.0,1.0,361,21,3,24,3.0,0,0,3,32:28:00,3,24,9,16,0,1,outdoors,grass,73,49.0,0,7,13,0
2,2,2002,1,200209080mia,det,0,36,17,220.0,206,1.0,19,51.0,2.0,257,15,3,14,1.0,2,1,2,25:36:00,3,14,7,14,0,1,outdoors,grass,87,69.0,0,12,21,0
3,3,2002,1,200209080mia,mia,1,27,18,207.0,207,3.0,41,182.0,3.0,389,27,0,0,0.0,2,0,0,34:24:00,0,0,6,12,1,1,outdoors,grass,87,69.0,0,12,49,1
4,4,2002,1,200209080jax,clt,0,31,19,211.0,203,3.0,28,104.0,0.0,307,18,1,8,0.0,2,2,2,27:27:00,1,8,9,14,0,0,outdoors,grass,85,0.0,0,16,28,1


In [5]:
data = df.select_dtypes(include=[object])
df.head(3)

,Unnamed: 0,season,week,boxscore_id,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,rush_attempts,rush_yards,rush_tds,total_yards,first_downs,sacks,sacks_yards,pass_interceptions,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,roof,surface,temp,humidity,wind_chill,wind_mph,Score,Won
0,0,2002,1,200209050nyg,sfo,0,26,16,166.0,166,1.0,25,113.0,0.0,279,13,0,0,1.0,0,0,1,27:32:00,0,0,4,12,0,0,outdoors,grass,73,49.0,0,7,16,1
1,1,2002,1,200209050nyg,nyg,1,45,28,342.0,318,0.0,22,43.0,1.0,361,21,3,24,3.0,0,0,3,32:28:00,3,24,9,16,0,1,outdoors,grass,73,49.0,0,7,13,0
2,2,2002,1,200209080mia,det,0,36,17,220.0,206,1.0,19,51.0,2.0,257,15,3,14,1.0,2,1,2,25:36:00,3,14,7,14,0,1,outdoors,grass,87,69.0,0,12,21,0


In [6]:
#dropping weather realted features
df_2=df.drop(['boxscore_id','roof', 'surface', 'temp','humidity','wind_chill','wind_mph'],axis=1)
df_2.head(1)

,Unnamed: 0,season,week,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,rush_attempts,rush_yards,rush_tds,total_yards,first_downs,sacks,sacks_yards,pass_interceptions,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,Score,Won
0,0,2002,1,sfo,0,26,16,166.0,166,1.0,25,113.0,0.0,279,13,0,0,1.0,0,0,1,27:32:00,0,0,4,12,0,0,16,1


In [7]:
df_2.shape

(9078, 30)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#df_2['team']=df_2['team'].tolist()
df_2['team'] = le.fit_transform(df_2['team'])

df_2['team']



df_2['time_of_possession'] = le.fit_transform(df_2['time_of_possession'])
df_2['time_of_possession']

0       495 
1       791 
2       379 
3       907 
4       490 
5       796 
6       734 
7       552 
8       412 
9       874 
10      659 
11      627 
12      698 
13      1164
14      119 
15      1181
16      1108
17      178 
18      379 
19      907 
20      595 
21      691 
22      1247
23      732 
24      352 
25      934 
26      749 
27      537 
28      408 
29      878 
       ...  
9048    879 
9049    407 
9050    140 
9051    1146
9052    535 
9053    751 
9054    1230
9055    68  
9056    717 
9057    569 
9058    353 
9059    933 
9060    863 
9061    423 
9062    691 
9063    595 
9064    82  
9065    1213
9066    270 
9067    1016
9068    146 
9069    1140
9070    176 
9071    1110
9072    799 
9073    684 
9074    1308
9075    111 
9076    453 
9077    833 
Name: time_of_possession, Length: 9078, dtype: int32

In [9]:
df_2.head(5)

,Unnamed: 0,season,week,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,rush_attempts,rush_yards,rush_tds,total_yards,first_downs,sacks,sacks_yards,pass_interceptions,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,Score,Won
0,0,2002,1,29,0,26,16,166.0,166,1.0,25,113.0,0.0,279,13,0,0,1.0,0,0,1,495,0,0,4,12,0,0,16,1
1,1,2002,1,19,1,45,28,342.0,318,0.0,22,43.0,1.0,361,21,3,24,3.0,0,0,3,791,3,24,9,16,0,1,13,0
2,2,2002,1,10,0,36,17,220.0,206,1.0,19,51.0,2.0,257,15,3,14,1.0,2,1,2,379,3,14,7,14,0,1,21,0
3,3,2002,1,15,1,27,18,207.0,207,3.0,41,182.0,3.0,389,27,0,0,0.0,2,0,0,907,0,0,6,12,1,1,49,1
4,4,2002,1,6,0,31,19,211.0,203,3.0,28,104.0,0.0,307,18,1,8,0.0,2,2,2,490,1,8,9,14,0,0,28,1


In [10]:
df_2=df_2.fillna(0)

In [11]:
df_2.head()

,Unnamed: 0,season,week,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,rush_attempts,rush_yards,rush_tds,total_yards,first_downs,sacks,sacks_yards,pass_interceptions,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,Score,Won
0,0,2002,1,29,0,26,16,166.0,166,1.0,25,113.0,0.0,279,13,0,0,1.0,0,0,1,495,0,0,4,12,0,0,16,1
1,1,2002,1,19,1,45,28,342.0,318,0.0,22,43.0,1.0,361,21,3,24,3.0,0,0,3,791,3,24,9,16,0,1,13,0
2,2,2002,1,10,0,36,17,220.0,206,1.0,19,51.0,2.0,257,15,3,14,1.0,2,1,2,379,3,14,7,14,0,1,21,0
3,3,2002,1,15,1,27,18,207.0,207,3.0,41,182.0,3.0,389,27,0,0,0.0,2,0,0,907,0,0,6,12,1,1,49,1
4,4,2002,1,6,0,31,19,211.0,203,3.0,28,104.0,0.0,307,18,1,8,0.0,2,2,2,490,1,8,9,14,0,0,28,1


In [12]:
# Step 0: Reformat data
data = df_2.values
X = data[:, 1:29]
y = data[:, 29]

In [13]:
X
X.shape

(9078, 28)

In [14]:
X

array([[2.002e+03, 1.000e+00, 2.900e+01, ..., 0.000e+00, 0.000e+00,
        1.600e+01],
       [2.002e+03, 1.000e+00, 1.900e+01, ..., 0.000e+00, 1.000e+00,
        1.300e+01],
       [2.002e+03, 1.000e+00, 1.000e+01, ..., 0.000e+00, 1.000e+00,
        2.100e+01],
       ...,
       [2.018e+03, 2.000e+01, 1.400e+01, ..., 0.000e+00, 0.000e+00,
        3.100e+01],
       [2.018e+03, 2.100e+01, 2.500e+01, ..., 0.000e+00, 0.000e+00,
        3.000e+00],
       [2.018e+03, 2.100e+01, 1.800e+01, ..., 0.000e+00, 1.000e+00,
        1.300e+01]])

In [15]:
scaled = preprocessing.MinMaxScaler()

In [16]:
scaled_X = scaled.fit_transform(X)

In [17]:
scaled_X

array([[0.        , 0.        , 0.93548387, ..., 0.        , 0.        ,
        0.25806452],
       [0.        , 0.        , 0.61290323, ..., 0.        , 0.14285714,
        0.20967742],
       [0.        , 0.        , 0.32258065, ..., 0.        , 0.14285714,
        0.33870968],
       ...,
       [1.        , 0.95      , 0.4516129 , ..., 0.        , 0.        ,
        0.5       ],
       [1.        , 1.        , 0.80645161, ..., 0.        , 0.        ,
        0.0483871 ],
       [1.        , 1.        , 0.58064516, ..., 0.        , 0.14285714,
        0.20967742]])

In [18]:
y

array([1., 0., 0., ..., 0., 0., 1.])

In [19]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, random_state=42)

In [20]:
X_train.shape

(6808, 28)

In [109]:
#Model1:XGB
# fit model no training data
xgb_model = make_pipeline(StandardScaler(),xgb.XGBClassifier())
xgb_model.fit(X_train, y_train)
print(f'LR_Accuracy: {xgb_model.score(X_test, y_test)} \n')
print(classification_report(y_test, xgb_model.predict(X_test)))
filename1 = 'xgb_model.sav'
joblib.dump(xgb_model, filename1)


LR_Accuracy: 0.8462555066079295 

              precision    recall  f1-score   support

         0.0       0.85      0.84      0.85      1136
         1.0       0.84      0.85      0.85      1134

   micro avg       0.85      0.85      0.85      2270
   macro avg       0.85      0.85      0.85      2270
weighted avg       0.85      0.85      0.85      2270



['xgb_model.sav']

In [110]:
#Model2:logisticRegression
#%%time

model_log_reg = make_pipeline(StandardScaler(),LogisticRegression())

model_log_reg.fit(X_train, y_train)

print(f'LR_Accuracy: {model_log_reg.score(X_test, y_test)} \n')
print(classification_report(y_test, model_log_reg.predict(X_test)))
filename2 = 'LR_model.sav'
joblib.dump(model_log_reg, filename2)



LR_Accuracy: 0.8563876651982378 

              precision    recall  f1-score   support

         0.0       0.87      0.84      0.85      1136
         1.0       0.85      0.87      0.86      1134

   micro avg       0.86      0.86      0.86      2270
   macro avg       0.86      0.86      0.86      2270
weighted avg       0.86      0.86      0.86      2270



C:\ProgramData\Anaconda3\envs\ut_ml\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['LR_model.sav']

In [111]:
#Model3:SVC
#%%time
clf = svm.SVC(gamma='scale',probability=True)
clf.fit(X_train,y_train)
# model_svm = make_pipeline(StandardScaler(),clf())
# model_svm.fit(X_train, y_train)

print(f'SVC_Accuracy: {clf.score(X_test, y_test)} \n')
print(classification_report(y_test, clf.predict(X_test)))
filename3 = 'svm_model.sav'
joblib.dump(clf, filename3)



SVC_Accuracy: 0.8466960352422908 

              precision    recall  f1-score   support

         0.0       0.86      0.83      0.84      1136
         1.0       0.84      0.86      0.85      1134

   micro avg       0.85      0.85      0.85      2270
   macro avg       0.85      0.85      0.85      2270
weighted avg       0.85      0.85      0.85      2270



['svm_model.sav']

In [112]:
%pwd

'C:\\Users\\Mrinalini\\Desktop\\NFL_Project\\NFL_ML_Predictor'

In [113]:
#Model4 : Gradient Boosting regressor fit with grid search
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier

In [114]:
GB_model = ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=10, n_estimators=1000,
                                               min_samples_split=2, min_samples_leaf=10, max_depth=4, min_impurity_decrease=0.0,                                               
                                               min_impurity_split=None, init=None, random_state=None, max_features=0.1, verbose=0)

In [115]:
#GB_model = make_pipeline(StandardScaler(),model())

GB_model.fit(X_train, y_train)

print(f'GB_Accuracy: {GB_model.score(X_test, y_test)} \n')
print(classification_report(y_test, GB_model.predict(X_test)))
filename4 = 'GB_model.sav'
joblib.dump(GB_model, filename4)

GB_Accuracy: 0.5484581497797357 

              precision    recall  f1-score   support

         0.0       0.75      0.15      0.24      1136
         1.0       0.53      0.95      0.68      1134

   micro avg       0.55      0.55      0.55      2270
   macro avg       0.64      0.55      0.46      2270
weighted avg       0.64      0.55      0.46      2270



['GB_model.sav']

In [116]:
#Gridsearch for best Hyperparameters
# from sklearn import model_selection
# from sklearn.model_selection import GridSearchCV
# param_grid={
#     'n_estimators':[500,1000,3000],
#     'max_depth':[4,6,8,10,20],
#     'learning_rate':[0.1,0.05,0.02,0.01],
#     'max_features':[1.0,0.3,0.1],
#     'loss':['deviance','exponential']
# }

In [117]:
# gs_cv = GridSearchCV(GB_model,param_grid,n_jobs=4)

In [118]:

# gs_cv.fit(X_train,y_train)

In [119]:
# print(gs_cv.best_params_)

In [120]:
GB_best_model = ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.02, n_estimators=500,
                                               min_samples_split=2, min_samples_leaf=10, max_depth=4, min_impurity_decrease=0.0,                                               
                                               min_impurity_split=None, init=None, random_state=None, max_features=0.3, verbose=0)

In [121]:
GB_best_model.fit(X_train, y_train)

print(f'GB_best_Accuracy: {GB_best_model.score(X_test, y_test)} \n')
print(classification_report(y_test, GB_best_model.predict(X_test)))
filename5 = 'GB_best_model.sav'
joblib.dump(GB_best_model, filename5)

GB_best_Accuracy: 0.8480176211453745 

              precision    recall  f1-score   support

         0.0       0.85      0.84      0.85      1136
         1.0       0.84      0.85      0.85      1134

   micro avg       0.85      0.85      0.85      2270
   macro avg       0.85      0.85      0.85      2270
weighted avg       0.85      0.85      0.85      2270



['GB_best_model.sav']

In [122]:
#In the last I have fit a Grid searched Gradient Boosting model 
#with best params which again gives an accuracy of 85% like most of above models.
#We may have to train more models-Deep Neural Net and CNN may be tried.

In [128]:
Xnew=[[0,0,0,29, 0, 20, 15, 130, 151, 1, 19, 110, 0, 181, 9, 0, 0, 1, 0, 0, 1, 495, 0, 0, 4, 7, 0,0 ]]

In [129]:
svm_model=joblib.load('./svm_model.sav')

In [130]:
result=svm_model.predict(Xnew)

In [131]:
result

array([0.])

In [132]:
logit_model=joblib.load('./LR_model.sav')

In [133]:
LR_result=logit_model.predict(Xnew)

In [134]:
print(LR_result)

[0.]


In [135]:
y_predicted_proba = logit_model.predict_proba(Xnew)

In [136]:
print(y_predicted_proba)

[[1. 0.]]


In [137]:
LR_result=logit_model.predict(X_test)

In [138]:
X_test

array([[0.125     , 0.55      , 0.70967742, ..., 0.        , 0.        ,
        0.43548387],
       [0.9375    , 0.1       , 0.83870968, ..., 0.        , 0.14285714,
        0.11290323],
       [0.0625    , 0.35      , 0.41935484, ..., 0.        , 0.14285714,
        0.27419355],
       ...,
       [0.6875    , 0.4       , 0.80645161, ..., 0.        , 0.14285714,
        0.33870968],
       [0.5625    , 0.        , 0.58064516, ..., 0.        , 0.        ,
        0.61290323],
       [0.6875    , 0.2       , 0.48387097, ..., 0.2       , 0.14285714,
        0.37096774]])

In [139]:
print(LR_result)

[1. 0. 0. ... 0. 1. 0.]


In [140]:
y_predicted_proba = logit_model.predict_proba(X_test)

In [141]:
print(y_predicted_proba)

[[0.0337065  0.9662935 ]
 [0.98616624 0.01383376]
 [0.98997762 0.01002238]
 ...
 [0.53313613 0.46686387]
 [0.23008645 0.76991355]
 [0.78373387 0.21626613]]


In [142]:
len(y_predicted_proba)

2270

In [143]:
len(X_test)

2270

In [144]:
logit_model.classes_

array([0., 1.])

In [145]:
#The predicted probabilities are for class 0(Loss in our case) and class 1(Win) respectively in each binary list

In [146]:
score=logit_model.score(X_test,y_test)
score

0.8563876651982378

In [147]:
from sklearn import metrics
cm=metrics.confusion_matrix(y_test,LR_result)

In [148]:
print(cm)

[[959 177]
 [149 985]]


In [149]:
SVM_result=svm_model.predict(X_test)

In [150]:
print(SVM_result)

[1. 0. 0. ... 1. 1. 0.]


In [151]:
y_predicted_proba1 = svm_model.predict_proba(X_test)

In [152]:
print(y_predicted_proba1)

[[0.01197706 0.98802294]
 [0.98925694 0.01074306]
 [0.99282229 0.00717771]
 ...
 [0.35970397 0.64029603]
 [0.45587744 0.54412256]
 [0.81136245 0.18863755]]


In [153]:
dta=pd.DataFrame(df_2)
print(dta.columns)

Index(['Unnamed: 0', 'season', 'week', 'team', 'home', 'pass_attempts',
       'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds',
       'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs',
       'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost',
       'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards',
       'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts',
       'fourth_down_conversions', 'Score', 'Won'],
      dtype='object')


In [154]:
xgb_model=joblib.load('./xgb_model.sav')

In [155]:
xgb_model_results=xgb_model.predict(X_test)

In [156]:
print(xgb_model_results)

[1. 0. 0. ... 1. 1. 0.]


In [157]:
y_predicted_proba2 = xgb_model.predict_proba(X_test)

In [158]:
print(y_predicted_proba2)

[[0.06256932 0.9374307 ]
 [0.9849413  0.0150587 ]
 [0.9485897  0.0514103 ]
 ...
 [0.3172409  0.6827591 ]
 [0.2741943  0.7258057 ]
 [0.8074151  0.1925849 ]]


In [159]:
GB_best_model=joblib.load('./GB_best_model.sav')
GB_best_model.predict(X_test)

array([1., 0., 0., ..., 1., 1., 0.])

In [160]:
y_predicted_proba3 = GB_best_model.predict_proba(X_test)

In [161]:
print(y_predicted_proba3)

[[0.05151984 0.94848016]
 [0.98373107 0.01626893]
 [0.94916993 0.05083007]
 ...
 [0.26710273 0.73289727]
 [0.19347261 0.80652739]
 [0.80655987 0.19344013]]


In [163]:
models=(xgb_model,logit_model,svm_model,GB_best_model)

In [166]:
#loop through 4 models and print results and probas
def predict_y(model,X):
    predicted_y=model.predict(X),
    y_predicted_proba=model.predict_proba(X)
    return print(predicted_y)
    return print(y_predicted_proba)
    

In [185]:
Logit_result=predict_y(logit_model,X_test)

(array([1., 0., 0., ..., 0., 1., 0.]),)


In [188]:
result_df=pd.DataFrame(Logit_result)

In [189]:
result_df

""
